In [1]:
import numpy as np
import collections
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import datetime  
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)
data_path = '../data-unversioned/p1ch7/'

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [3]:
cifar10 = datasets.CIFAR10(data_path, train=True, download= True)           # Training dataset....
cifar10_val = datasets.CIFAR10(data_path, train=False, download= True)      # Dataset for validation....

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'] # class definations...

In [5]:
# Splitting dataset into training and validation set...

# Trainig data...
transformed_cifar10_Train = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Validation data...
transformed_cifar10_Val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [6]:
# Class defining the ResBlock...
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x


# Class "ResNet-10" for model defination with 10 ResBlocks...
class ResNet10BN(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        # Batch Norm part...
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_chans1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        # Batch Norm part...
        out = F.max_pool2d(torch.relu(self.conv1_batchnorm(self.conv1(x))), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

# Traing loop function...
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader:  
            imgs = imgs.to(device=device)  
            labels = labels.to(device=device)
            outputs = model(imgs)  
            loss = loss_fn(outputs, labels)  
            optimizer.zero_grad()         
            loss.backward()         
            optimizer.step()  
            loss_train += loss.item()  

        if epoch == 1 or epoch % 30 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader))) 

# Function calculating the validation of the model...
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad(): 
            for imgs, labels in loader:
                imgs = imgs.to(device=device)  
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0]  
                correct += int((predicted == labels).sum())  

        print("Accuracy {}: {:.2f}".format(name , correct / total))

In [ ]:
# Training the model "ResNet-10" with dataset divided into minibatches...
train_mbload = torch.utils.data.DataLoader(transformed_cifar10_Train, batch_size=64, shuffle=True) 

model = ResNet10BN(n_chans1=32, n_blocks=10).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_mbload,
)

2022-03-31 00:42:22.012548 Epoch 1, Training loss 1.7120192020445528
2022-03-31 01:02:59.712020 Epoch 30, Training loss 0.610275738227093
2022-03-31 01:24:02.345697 Epoch 60, Training loss 0.38638055659925846
2022-03-31 01:45:00.784462 Epoch 90, Training loss 0.23001980790130014
2022-03-31 02:06:00.324907 Epoch 120, Training loss 0.12215372643021442
2022-03-31 02:26:58.429140 Epoch 150, Training loss 0.0696690642891828


In [ ]:
# Checking the accuracy of model...
train_accuload = torch.utils.data.DataLoader(transformed_cifar10_Train, batch_size = 64, shuffle=False)
Valid_accuload = torch.utils.data.DataLoader(transformed_cifar10_Val, batch_size = 64, shuffle=False)

validate(model, train_accuload , Valid_accuload)